In [1]:
import numpy as np
import pandas as pd
import os
import tempfile

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from os import listdir
from os.path import isfile, join
path_train = 'ignored_assets/public_data/training_data/order_data'
trainfiles = [f for f in listdir(path_train) if isfile(join(path_train, f)) and f != '.DS_Store']

path_test1 = 'ignored_assets/public_data/test_set_1/order_data'
testfiles1 = [f for f in listdir(path_test1) if isfile(join(path_test1, f)) and f != '.DS_Store']

path_test2 = 'ignored_assets/public_data/test_set_2/order_data'
testfiles2 = [f for f in listdir(path_test2) if isfile(join(path_test2, f)) and f != '.DS_Store']


In [3]:
orderdf = []
for f in trainfiles:
    orderdf.append(pd.read_csv(join(path_train, f), delimiter='\t', header = None))
    
for f in testfiles1:
    orderdf.append(pd.read_csv(join(path_test1, f), delimiter='\t', header = None))
    
for f in testfiles2:
    orderdf.append(pd.read_csv(join(path_test2, f), delimiter='\t', header = None))




In [4]:
orderdf = pd.concat(orderdf)

In [5]:
#Clean up
orderdf[6] = pd.to_datetime(orderdf[6])
orderdf.columns = ['order_id', 'driver_id', 'passenger_id', 'start_district_hash', 'dest_district_hash', 'price', 'timestamp']


In [7]:
driver_ids=orderdf['driver_id'].unique()

array(['dd65fa250fca2833a3a8c16d2cf0457c',
       'c077e0297639edcb1df6189e8cda2c3d',
       '86dbc1b68de435957c61b5a523854b69', ...,
       'd93e74504885e525cf24a64e3b7157dd',
       '3f84d9a739831f50c4a629f5d94f7fc2',
       '6c01b3c1bd5b7e2c9d7deb5b419a6608'], dtype=object)

In [76]:
def count_empty_trips(driver_hash):
    #Copy out driver info
    driver_orders=orderdf[orderdf['driver_id']==driver_hash]
    driver_orders=driver_orders.sort_values('timestamp')
    
    #Auxiliary stuff
    driver_orders['next_timestamp']=driver_orders['timestamp'].shift(-1)
    driver_orders['next_start_district_hash']=driver_orders['start_district_hash'].shift(-1)
    break_threshold=pd.Timedelta('1 hour')
    
    #Did the driver go on break after the ride? Did the driver rebalance?
    driver_orders['break']=(driver_orders['next_timestamp']-driver_orders['timestamp'])>break_threshold
    driver_orders['emptytrip']=(driver_orders['dest_district_hash']!=driver_orders['next_start_district_hash'])
    
    if len(driver_orders)>0:
        return float(sum(driver_orders['emptytrip'] & ~driver_orders['break']))/float(len(driver_orders))
    else:
        return np.NaN

In [79]:
count_empty_trips(driver_ids[2])

0.18796992481203006

In [69]:
driver_pd=pd.DataFrame(driver_ids,columns=['driver_id'])

In [ ]:
driver_pd['frac_reb']=driver_pd.apply(lambda x: count_empty_trips(x['driver_id']),axis=1)

0.24916943521594684

In [70]:
driver_pd

,driver_id
0,dd65fa250fca2833a3a8c16d2cf0457c
1,c077e0297639edcb1df6189e8cda2c3d
2,86dbc1b68de435957c61b5a523854b69
3,4fadfa6eeaa694742de036dddf02b0c4
4,NaN
5,115ac9b23f00a2e6d8a3041e23469f41
6,7cb7ccbc192c177b136feca274fb3d07
7,c586e492c4587c3aa15557a8eb41806d
8,ea61e970ba415f946b3b35464c74f32a
9,adabcd23118a35be7cc03c8f60d73e2e


In [61]:
count_empty_trips(driver_ids[0])

0.20689655172413793

In [ ]:
#Create a timeline of rebalancing trips.
# For each, record two timestamps (last pickup, next pickup), start location, and end location.
# Sketch: for each driver, return a list of reb trips by masking. Then append.
# Can we do this in a non-sequential way?

In [ ]:
def find_empty_trips(driver_hash):
    #Copy out driver info
    driver_orders=orderdf[orderdf['driver_id']==driver_hash]
    driver_orders=driver_orders.sort_values('timestamp')
    
    #Auxiliary stuff
    driver_orders['next_timestamp']=driver_orders['timestamp'].shift(-1)
    driver_orders['next_start_district_hash']=driver_orders['start_district_hash'].shift(-1)
    break_threshold=pd.Timedelta('1 hour')
    
    #Did the driver go on break after the ride? Did the driver rebalance?
    driver_orders['break']=(driver_orders['next_timestamp']-driver_orders['timestamp'])>break_threshold
    driver_orders['emptytrip']=(driver_orders['dest_district_hash']!=driver_orders['next_start_district_hash'])
    
    return driver_orders[driver_orders['emptytrip'] & ~driver_orders['break']]

In [ ]:
empty_rides_list=driver_pd.apply(lambda x: find_empty_trips(x['driver_id']),axis=1)